In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import densenet

In [ ]:
!echo $PYTHONPATH

In [ ]:
%env PYTHONPATH=/home/jinen/git/Pneumonia-Classification/task-3-modeling/src/

In [ ]:
os.getcwd()

In [ ]:
import os
import mlflow
from task2.data_processing import get_data_generators ## imports using ln -sf 
from base.const.general_const import NOTEBOOK, IMG_SIZE, IMG_SHAPE, CLASS_NAME_PATH,\
    CLASS_MODE, BATCH_SIZE, PROD_MODEL_PATH, PROJECT_NAME
from const.modeling_const import PROCESSED_TRAIN_PATH, PROCESSED_TEST_PATH, INIT_EPOCHS, LEARNING_RATE, CSV_LOG_PATH, CHECKPOINT_PATH, DH_LOG_PARAM_PATH
from .training import build_model, get_callbacks
from dagshub import dagshub_logger

In [ ]:
DAGSHUB_USER_NAME = input("DAGsHub Username:")
DAGSHUB_TOKEN = input(f"Token for {DAGSHUB_USER_NAME}:")
DAGSHUB_REPO_NAME = "Pneumonia-Classification"
DAGSHUB_REPO_OWNER = "nirbarazida"

In [ ]:
os.environ['MLFLOW_TRACKING_USERNAME'] = DAGSHUB_USER_NAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = DAGSHUB_TOKEN

mlflow.set_tracking_uri(f'https://dagshub.com/{DAGSHUB_REPO_OWNER}/{DAGSHUB_REPO_NAME}.mlflow')

training_generator, validation_generator, test_generator = get_data_generators(DATA_DIR='/home/jinen/git/Pneumonia-Classification/data/',
                                                                               PROJECT_NAME=PROJECT_NAME,
                                                                               BATCH_SIZE=BATCH_SIZE,
                                                                               DIM=IMG_SIZE)

In [ ]:
model = build_model(IMG_SHAPE, LEARNING_RATE)
csv_logger, early_stopping, model_checkpoint = get_callbacks(CSV_LOG_PATH, CHECKPOINT_PATH)
history = model.fit(training_generator,
                    epochs=INIT_EPOCHS,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    callbacks=[csv_logger, early_stopping, model_checkpoint])

In [ ]:
with dagshub_logger(should_log_metrics=False,hparams_path=DH_LOG_PARAM_PATH) as logger:
    logger.log_hyperparams(model.history.params)

model.save(PROD_MODEL_PATH)